In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [1]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'module1-statistics-probability-and-inference'))
	# print(os.getcwd())
except:
	pass

 <img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
 <br></br>
 <br></br>

 ## *Data Science Unit 1 Sprint 3 Assignment 1*

 # Apply the t-test to real data

 Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

 Your goals:

 1. Load and clean the data (or determine the best method to drop observations when running tests)
 2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
 3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
 4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

 Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

 Stretch goals:

 1. Refactor your code into functions so it's easy to rerun with arbitrary variables
 2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [2]:
### YOUR CODE STARTS HERE
import pandas
import numpy

cols = ['party','handicapped-infants','water-project',
		'budget','physician-fee-freeze', 'el-salvador-aid',
		'religious-groups','anti-satellite-ban',
		'aid-to-contras','mx-missile','immigration',
		'synfuels', 'education', 'right-to-sue','crime','duty-free',
		'south-africa']
get_ipython().system('wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data')


df = pandas.read_csv('house-votes-84.data',
               header=None,
               names=cols).replace({'?':numpy.NaN, 'n':0, 'y':1})
df.head()


--2019-09-16 13:34:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Server file no newer than local file ‘house-votes-84.data’ -- not retrieving.



,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [3]:
import scipy.stats

democrats = df[df['party']=='democrat']
republicans = df[df['party']=='republican']
for column in cols[1:]:
	result = scipy.stats.ttest_ind(republicans[column], democrats[column], nan_policy='omit')
	demPercent = democrats[column].mean()*100
	repPercent = republicans[column].mean()*100
	if result[1] < 0.05:
		print(f'Due to a p-value of {result[1]:.6} < 0.05, we reject the null hypothesis that democrat ({demPercent:.4}% in favor) and republican ({repPercent:.4}% in favor) house members vote similarly on {column}.')
	else:
		print(f'Due to a p-value of {result[1]:6.6} > 0.05, we fail to reject the null hypothesis that democrat and republican house members vote similarly on {column}.')


Due to a p-value of 1.61344e-18 < 0.05, we reject the null hypothesis that democrat (60.47% in favor) and republican (18.79% in favor) house members vote similarly on handicapped-infants.
Due to a p-value of 0.929156 > 0.05, we fail to reject the null hypothesis that democrat and republican house members vote similarly on water-project.
Due to a p-value of 2.07034e-77 < 0.05, we reject the null hypothesis that democrat (88.85% in favor) and republican (13.41% in favor) house members vote similarly on budget.
Due to a p-value of 1.99426e-177 < 0.05, we reject the null hypothesis that democrat (5.405% in favor) and republican (98.79% in favor) house members vote similarly on physician-fee-freeze.
Due to a p-value of 5.60052e-68 < 0.05, we reject the null hypothesis that democrat (21.57% in favor) and republican (95.15% in favor) house members vote similarly on el-salvador-aid.
Due to a p-value of 2.39367e-20 < 0.05, we reject the null hypothesis that democrat (47.67% in favor) and republ

In [4]:
# Extra 1-sample t-testing added during lecture

result = scipy.stats.ttest_1samp(democrats['synfuels'], 0.5, nan_policy='omit')
demPercent = democrats['synfuels'].mean()*100
print(f'Due to a p-value of {result[1]:.6} > 0.05, we fail to reject the null hypothesis that democrat house members ({demPercent:.4}% in favor) do not vote significantly in favor or against synfuels.')
result = scipy.stats.ttest_1samp(republicans['synfuels'], 0.5, nan_policy='omit')
repPercent = republicans['synfuels'].mean()*100
print(f'Due to a p-value of {result[1]:.6} < 0.05, we reject the null hypothesis that republican house members ({repPercent:.4}% in favor) do not vote significantly in favor or against synfuels.')



Due to a p-value of 0.851409 > 0.05, we fail to reject the null hypothesis that democrat house members (50.59% in favor) do not vote significantly in favor or against synfuels.
Due to a p-value of 1.51577e-28 < 0.05, we reject the null hypothesis that republican house members (13.21% in favor) do not vote significantly in favor or against synfuels.
